In [35]:
import os

os.chdir("/home/tales/dev/master/mdc_analysis/")
print("working dir", os.getcwd())

import pandas as pd
import gc

from bokeh.io import output_notebook, show
output_notebook()

from src.dao import csv_dao
from src.entity.stop_region import StopRegion, StopRegionGroup, sr_row_to_stop_region
from src.entity.stop_region import group_stop_regions_for_agglutination, same_closest_poi
from src.poi_grabber import google_places, hot_osm
from src.plot import plot
from src.utils import geo
from src.utils.others import concat_lists

# pd.set_option('display.float_format', lambda x: '%.4f' % x)

# import warnings
# warnings.filterwarnings('ignore')

# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

working dir /home/tales/dev/master/mdc_analysis


Loading BokehJS ...

In [2]:
user_id = pd.Series(csv_dao.list_stop_region_usernames()).sample().item()
user_id = 6070

sr_list = csv_dao.stop_region_sequence(user_id)
print("Stop Regions sequence size: {}".format(len(sr_list)))

Stop Regions sequence size: 230


In [3]:
# place_ids = []

# for sr in sr_list:
#     sr.load_close_pois()
#     place_ids.append(sr.close_pois.index)

# Stop Region

In [4]:
one_sr = sr_list[22]
one_sr.sr_id

'6070_22'

#### Tags closest POI

In [5]:
one_sr.tag_closest_poi()

[['store']]

#### Tags of close POIs

In [6]:
one_sr.tag_radius_pois(radius=10)

[['store'],
 ['clothing_store', 'store'],
 ['store'],
 ['atm', 'finance'],
 ['shoe_store', 'store'],
 ['electronics_store', 'home_goods_store', 'store'],
 ['supermarket', 'grocery_or_supermarket', 'food', 'store'],
 ['insurance_agency']]

#### Close POIs

In [7]:
one_sr.closest_poi()

,name,place_id,plus_code,price_level,rating,types,user_ratings_total,vicinity,latitude,longitude,distance,position,sr_id
17,mobilezone,ChIJ0w1BnV0SjkcR79yP62Ym_bY,"{'compound_code': 'R532+6R Fribourg, Switzerla...",NaN,4.0,[store],13.0,"Avenue de la Gare 10, Fribourg",46.803008,7.152083,4.773307,0,6070_22


In [8]:
one_sr.radius_pois(radius=7)

,name,place_id,plus_code,price_level,rating,types,user_ratings_total,vicinity,latitude,longitude,distance,position,sr_id
17,mobilezone,ChIJ0w1BnV0SjkcR79yP62Ym_bY,"{'compound_code': 'R532+6R Fribourg, Switzerla...",NaN,4.0,[store],13.0,"Avenue de la Gare 10, Fribourg",46.803008,7.152083,4.773307,0,6070_22
14,Import Parfumerie Fribourg,ChIJX25cMSxpjkcRWuhOtbKtx8M,"{'compound_code': 'R532+6V Fribourg, Switzerla...",NaN,4.7,"[clothing_store, store]",3.0,"Avenue de la Gare 12, Fribourg",46.803063,7.152141,5.609153,1,6070_22


# Stop Region AGGLUTINATIONS

There are some consecutive Stop Regions that could be understood as one

#### Some consecutive Stop Regions may refer to the same place although they are found different.<br>
This situation may occurr due to the process of discovering Stop Regions. <br>
The Stop Region is made by iterating consecutive GPS points that: <br>
1) While being iterated, they form a potential Stop Region cluster. If the next point lies at maximum 50m from this potential Stop Region cluster's centroid, the point is added to it.<br>
  1.1) If one GPS point happen to be out of this range it is still added to the cluster. <br>
  1.2) If two consecutive points happen to be out of this range, the potential Stop Region cluster formation is ended.<br>
  
2) When the potential Stop Region cluster formation is ended, is time to check if it can be considered as a Stop Region. This validation is made by calculating the delta time between the first and the last cluster points.


In [9]:
agglutinated_srs, agglut_report = group_stop_regions_for_agglutination(sr_list, same_closest_poi)

print("Interesting rows\n-----------------")
print("LOW delta_t, HIGH distance: {}".format("0"))
print("HIGH delta_t, LOW distance: {}".format("7, 8"))
print("Same place, not agglutinate: {}".format("199 to 207"))

agglut_report.iloc[185:200]

Interesting rows
-----------------
LOW delta_t, HIGH distance: 0
HIGH delta_t, LOW distance: 7, 8
Same place, not agglutinate: 199 to 207


,agglutinate,delta_t,distance,last_sr_tag,sr_tag,last_sr_semantics,sr_semantics,last_sr,sr
185,True,10.0,6.2,"[[health, local_government_office]]","[[health, local_government_office]]",[],[],6070_185,6070_186
186,True,280.0,12.3,"[[health, local_government_office]]","[[health, local_government_office]]",[],[],6070_186,6070_187
187,False,444.0,165.2,"[[health, local_government_office]]",[[school]],[],[],6070_187,6070_188
188,False,20.0,133.4,[[school]],[[embassy]],[],[],6070_188,6070_189
189,True,50.0,12.3,[[embassy]],[[embassy]],[],[],6070_189,6070_190
190,True,10.0,14.5,[[embassy]],[[embassy]],[],[],6070_190,6070_191
191,True,10.0,1.9,[[embassy]],[[embassy]],[],[],6070_191,6070_192
192,True,10.0,8.8,[[embassy]],[[embassy]],[],[],6070_192,6070_193
193,True,10.0,7.5,[[embassy]],[[embassy]],[],[],6070_193,6070_194
194,True,10.0,7.2,[[embassy]],[[embassy]],[],[],6070_194,6070_195


In [10]:
print("{} stop regions, before agglutination".format(len(sr_list)))
#p = StopRegionGroup(sr_list).plot()

230 stop regions, before agglutination


In [11]:
stop_regions_group = StopRegionGroup(sr_list).agglutinate_stop_regions()

print("{} stop regions".format(stop_regions_group.size()))
p = stop_regions_group.plot(fill_color="magenta", plot_n_pois=6)

164 stop regions


In [12]:
agg_report = stop_regions_group.sequence_report()
stop_regions_group.sequence_report().iloc[144:154]

,delta_t,distance,last_sr_type,sr_type,last_sr_semantics,sr_semantics,last_sr,sr
144,20.0,148.9,[[school]],[[embassy]],[],[],6070_188,agg_6070_189
145,40.0,23.4,[[embassy]],"[[health, local_government_office]]",[],[],agg_6070_189,agg_6070_196
146,434.0,20.1,"[[health, local_government_office]]",[[embassy]],[],[],agg_6070_196,agg_6070_198
147,70.0,30.8,[[embassy]],"[[health, local_government_office]]",[],[],agg_6070_198,6070_206
148,10.0,31.3,"[[health, local_government_office]]",[[embassy]],[],[],6070_206,agg_6070_207
149,222509.0,47.8,[[embassy]],"[[accounting, finance]]",[],[],agg_6070_207,6070_209
150,10.0,60.9,"[[accounting, finance]]","[[health, local_government_office]]",[],[],6070_209,6070_210
151,60.0,14.9,"[[health, local_government_office]]",[[embassy]],[],[],6070_210,agg_6070_211
152,20.0,16.3,[[embassy]],"[[health, local_government_office]]",[],[],agg_6070_211,6070_215
153,20.0,13.7,"[[health, local_government_office]]",[[embassy]],[],[],6070_215,agg_6070_216


Stop Region <b>agg_6070_133</b> seems to be in a Park... however park anotation is not the closest one <br>
Is it a good idea to remove parking POIs?

## Adding HOT OSM POIs to that location

In [13]:
case_sr = stop_regions_group.search_stop_region_by_id("agg_6070_133")

hot_osm_pois = hot_osm.load_hot_osm_pois(valid_pois=True)
hot_osm_pois["latitude"] = hot_osm_pois["lat_4326"]
hot_osm_pois["longitude"] = hot_osm_pois["lon_4326"]

osm_close = geo.slice_geo_data2(hot_osm_pois, case_sr.centroid_lat, case_sr.centroid_lon, search_tolerance=0.005)

print(len(osm_close))

/home/tales/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (3,4,6,8,9,11,13,15,16,19,20,24,25,27,28,29,32,33,34,35,37,39,40,41,43,47,50,52,53,54,56,57,59,60,63,64,65,66,69) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


20


In [14]:
from src.plot.plot2 import add_hot_osm_pois

p = add_hot_osm_pois(pois=osm_close, size=8, p=p)

It seems to be a good idea to go back here and consider also the HOT OSM POIs

In [15]:
def mark_home_and_work(plot, sr_group):
    for sr in stop_regions_group.stop_region_list:
        if "HOME" in sr.semantics:
            plot = sr.plot_simple(p=plot, color="lightblue", legend="HOME")

        if "WORK" in sr.semantics:
            plot = sr.plot_simple(p=plot, color="lightgreen", legend="WORK")
    return plot

In [16]:
p = mark_home_and_work(p, stop_regions_group)
show(p)

### Coding the tags sequence
Tags are POIs types and Stop Region Semantics
Semantics are more important than POIs types

In [17]:
r = stop_regions_group.sequence_report()
r.head(8)

,delta_t,distance,last_sr_type,sr_type,last_sr_semantics,sr_semantics,last_sr,sr
0,10.0,32.3,[[embassy]],"[[health, local_government_office]]",[],[],6070_0,6070_1
1,152920.0,26983.2,"[[health, local_government_office]]","[[liquor_store, store]]",[],[],6070_1,6070_2
2,6926.0,332.4,"[[liquor_store, store]]","[[bus_station, transit_station]]",[],[],6070_2,6070_3
3,2811.0,27682.9,"[[bus_station, transit_station]]",[[night_club]],[],[],6070_3,6070_4
4,737.0,847.1,[[night_club]],[[embassy]],[],[HOME],6070_4,6070_5
5,159469.0,26506.6,[[embassy]],"[[moving_company, storage]]",[HOME],[],6070_5,6070_6
6,2670.0,27211.7,"[[moving_company, storage]]",[[store]],[],[],6070_6,agg_6070_7
7,159.0,198.2,[[store]],"[[cafe, food]]",[],[],agg_6070_7,6070_10


In [29]:
tags = r[["sr_semantics", "sr_type"]].apply(lambda row : {"semantics": row["sr_semantics"], "types": row["sr_type"]}, axis=1)

In [37]:
tags.apply(lambda tag_dict : concat_lists(tag_dict["types"]) if len(tag_dict["semantics"]) == 0 else tag_dict["semantics"])

0                      [health, local_government_office]
1                                  [liquor_store, store]
2                         [bus_station, transit_station]
3                                           [night_club]
4                                                 [HOME]
5                              [moving_company, storage]
6                                                [store]
7                                           [cafe, food]
8                                                 [HOME]
9         [meal_takeaway, restaurant, food, art_gallery]
10                                   [shoe_store, store]
11                        [bus_station, transit_station]
12                                              [health]
13                        [cafe, food, store, hair_care]
14                                                [WORK]
15     [shopping_mall, restaurant, food, real_estate_...
16                                                [WORK]
17                             